# ImpDAR Getting Started Tutorial

Welcome to ImpDAR!
This is an impulse radar processor that we in UW ESS are building for our ice-penetrating radar systems. However, the potential applications of this software are much more widespread. Our goal is to provide an alternative to the expensive commercial software that would be purchased with a ground-penetrating radar system from a company such as GSSI or Sensors and Software. We provide all of the processing tools that are needed to move from a raw data file to an interpretable image of the subsurface. Moreover, our software is agnostic to the system used, so we can import data from a variety of different ground-penetrating radar systems and interact with the data in the exact same way.

You are running this Jupyter Notebook through a 'binder' where ImpDAR is installed, so you don't need to install anything locally on your machine. If, on the other hand, you want to get ImpDAR running locally, you can go to our github page (https://github.com/dlilien/ImpDAR) to clone the repository. 

The preferred pathway for interacting with ImpDAR is through the terminal. However, our API allows relatively easy access through other programs as well. Here, we want to walk you through the processing flow using a Jupyter Notebook, so all of the processing functions will be called through Python. No prior knowledge of Python is necessary.

Using the Jupyter Notebook:
- This format allows us to easily incorporate both explanatory text and python code.
- Text and code can both be edited.
- To run a code cell click inside it and either press the run button at the top or press ctrl+enter on your keyboard.
- The cells are meant to be run in order. Having said that, nothing will stop you from running the lower cells first. If an error comes up make sure that you have loaded all the proper libraries from the cells above.
- I encourage you to change a few of the coding cells. I generally write notes to the user with three ###, in some places telling you to change things and in others telling you not to. If you are familiar with Python feel free to change anything, even if it says not to.

### Import the necessary libraries

The very first thing to do with any Python script is to import all of the 'libraries' that will be used. These will allow us to call some lower level functions that help with loading scripts, some numerical issues, and plotting. Knowing exactly what each of these are is not important to understanding the radar processing flow. 

In [ ]:
### Don't Change ###

# Standard Python Libraries
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Load the raw data

With the standard libraries loaded, we can now look at some radar data. These are data collected by Knut at the Northeast Greenland Ice Stream. We will talk a little bit more about exactly what we are looking at when we get to an image that you will be able to interpret. 

In [ ]:
### Don't Change ###

# Impdar's loading function
from impdar.lib import load

# To look through files
import glob

# Look for the appropriate files and then load them as a data 'class'
files = glob.glob('./impdar/tests/input_data/negis_example_data/*[!.gps]')
dat = load.load('gecko',files)[0]

# Plot the raw data
plt.figure(figsize=(4,3))
plt.imshow(dat.data,cmap='Greys',aspect='auto',vmin=-10,vmax=10)
plt.ylabel('Sample Number')
plt.xlabel('Trace Number')
plt.tight_layout()
plt.show()

Yuck. Raw radar data are a mess. We can't really see anything meaningful here. 

In order to illustrate the first processing step, let's plot a single 'trace'. A radar trace is one collection of voltages measured by the oscilloscope through time. The total time for collection in this case is ~50 microseconds. 

In [ ]:
### Try changing the trace index to see whether they are similar or different ###
### Make sure that you understand the relationship between this figure and above ###

# Reload in case a lower cell gets run
dat = load.load('gecko',files)[0]

# Index the data to get one column from the matrix
trace = dat.data[:,600]

### Don't Change ###

# Plot a new figure for the single trace
plt.figure(figsize=(3,4))
plt.plot(trace,dat.travel_time)
plt.xlim(-5000,5000)
plt.gca().invert_yaxis()
plt.ylabel('Travel Time ($\mu$ sec)')
plt.xlabel('mV')
plt.tight_layout()
plt.show()

## Bandpass filter

The first big processing step is to isolate the frequency band of interest. The radar antennas set this frequency. In our case, the antennas are ~20 m long (frequency of 3 MHz) so we want to allow the frequency band from 1-5 MHz pass while damping all other frequencies (i.e. bandpass filter). 

This filter works on all the traces simultaneously, but to illustrate its effect we will plot the single trace again.

In [ ]:
### Change the frequency band here ###

# Do the vertical bandpass filter from 1-5 MHz
dat.vertical_band_pass(1,5)

### Don't Change Below ###

# Index the data to get one column from the matrix
trace = dat.data[:,600]

# Plot the trace
plt.figure(figsize=(3,4))
plt.plot(trace,dat.travel_time)
plt.xlim(-100,100)
plt.gca().invert_yaxis()
plt.ylabel('Travel Time ($\mu$ sec)')
plt.xlabel('mV')
plt.tight_layout()
plt.show()

Now plot the entire image again.

In [ ]:
# Plot all the bandpassed data
plt.figure(figsize=(6,4))
plt.imshow(dat.data,cmap='Greys',aspect='auto',vmin=-10,vmax=10,
          extent=[0,dat.tnum,max(dat.travel_time),min(dat.travel_time)])
plt.ylabel('Travel Time ($\mu$ sec)')
plt.xlabel('Trace Number')
plt.tight_layout()
plt.show()

Some questions to check your understanding so far:
- Describe the differences between the plotted trace before and after the bandpass filter.
- Knowing how the 'bandpass' filter works, how would you describe a 'highpass' or a 'lowpass' filter?
- We see a large return at ~30 microseconds. Knowing that the speed of light in ice is 169 million meters per second, what is the depth of this feature. What do you think it might be?

### Geometric Corrections

Now that the data actually look like something useful we can do a few more corrections to make it more physical. Two geometric corrections allow us to look at the image more like something under the surface instead of in this weird time dimension. First we crop out the 'pretrigger' which is data collected by the receiver before the radar pulse is actually transmitted. Second we do a 'normal move-out' correction (nmo) which corrects for the separation distance between the receiving and transmitting antennas.

In [ ]:
# Crop the pretrigger out from the top of each trace
dat.crop(0,dimension='pretrig')
# Do the normal move-out correction with antenna spacing 160 m
dat.nmo(160)

# Re-plot the image
plt.figure(figsize=(6,4))
plt.imshow(dat.data,cmap='Greys',aspect='auto',vmin=-10,vmax=10,
          extent=[0,dat.tnum,max(dat.nmo_depth),min(dat.nmo_depth)])
plt.ylabel('Depth (m)')
plt.xlabel('Trace Number')
plt.tight_layout()
plt.show()

### Georeferencing and Interpolation

Up to now, everything has been plotted as 'trace number' in the horizontal. In reality though, we want to know where these were measured. Typically, these data are collected with some kind of GPS. Lets look at the lat/long data now.

In [ ]:
# Plot the profile's location in Lat-Long space
plt.figure()
plt.scatter(dat.long,dat.lat,c=dat.trace_num)
plt.ylabel('Latitude')
plt.xlabel('Longitude')
plt.colorbar(label='Trace Number')
plt.show()

In [ ]:
# Convert from lat-long to x-y
from impdar.lib.gpslib import get_utm_conversion
transform = get_utm_conversion(np.nanmean(dat.lat),np.nanmean(dat.long))
pts = np.array(transform(np.vstack((dat.long,dat.lat)).transpose()))
x = pts[:,0]
y = pts[:,1]

# Calculate the cumulative distance travelled
distance_travelled = np.insert(np.cumsum(np.sqrt((x[1:]-x[:-1])**2.+(y[1:]-y[:-1])**2.)),0,0)

# Replot the points, now in x-y-distance
plt.figure()
plt.scatter(x,y,c=distance_travelled)
plt.colorbar(label='Distance Travelled (m)')

In [ ]:
# Plot the distance between traces
plt.plot(np.gradient(distance_travelled))

In [ ]:
# Interpolate onto equal trace spacing
from impdar.lib.gpslib import interp
interp([dat],5,guess_offset=False)

# Plot the new trace spacing (should be uniform after interpolation)
plt.figure()
plt.plot(np.gradient(dat.dist))

In [ ]:
# Plot the final figure
plt.figure(figsize=(6,4))
plt.imshow(dat.data,cmap='Greys',vmin=-10,vmax=10,aspect='auto',
          extent=[min(dat.dist),max(dat.dist),max(dat.nmo_depth),min(dat.nmo_depth)])
plt.ylabel('Depth (m)')
plt.xlabel('Distance (km)')
plt.tight_layout()
plt.show()

### Comments on additional processing steps

- Denoising
- Migration
- Power corrections

# ImpDAR Interpreter

In [ ]:
#from impdar.bin.imppick import pick

#pick(dat)